In [38]:
import numpy as np

In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModel.from_pretrained("distilgpt2")

Some weights of the model checkpoint at distilgpt2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
def get_features(text):
    inp = tokenizer(text, return_tensors="pt")
    outp = model(**inp)
    return outp.last_hidden_state.detach().cpu().mean(1).squeeze(0).numpy()

In [73]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset["train"].shuffle().select(range(1000))

In [80]:
from tqdm import tqdm
result = []
for item in tqdm(train_dataset):
    result.append(get_features(item["text"]))
train_x_features = np.array(result)

100%|██████████| 1000/1000 [00:17<00:00, 55.85it/s]


In [82]:
val_x_features = []
val_dataset = dataset["validation"].select(range(200))
val_y = val_dataset["label"]
for item in tqdm(val_dataset):
    val_x_features.append(get_features(item["text"]))

100%|██████████| 200/200 [00:03<00:00, 58.47it/s]


In [83]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_x_features = scaler.fit_transform(train_x_features)
val_x_features = scaler.transform(val_x_features)

In [84]:
train_y = train_dataset["label"]
# train logistic regression on X and Y
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=500).fit(train_x_features, train_y)


In [86]:
print(f"Precision: {100*np.mean(clf.predict(val_x_features) == val_y):.2f}%")

Precision: 71.50%
